<a href="https://colab.research.google.com/github/jetsonmom/6.23_automobility_lesson/blob/main/6_23_%EA%B0%95%EC%9D%98_step2_1%EB%8B%A8%EA%B3%84_%EC%A7%81%EC%84%A0%2C_%EA%B3%A1%EC%84%A0_%EC%B0%A8%EC%84%A0_%EB%94%B0%EB%9D%BC%EA%B0%80%EA%B8%B0_%EC%95%A0%EB%8B%88%EB%A9%94%EC%9D%B4%EC%85%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1단계: 간단한 차선 따라가기 애니메이션

In [ ]:
# 첫 번째 셀 - 라이브러리 설치
!pip install matplotlib
!pip install numpy
!pip install IPython

# 애니메이션이 안 되면 추가 설치
!apt-get update
!apt-get install -y ffmpeg

In [ ]:
# 🚗 1단계: 기본 차선 유지 시뮬레이션
# Google Colab에서 실행 가능한 애니메이션

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import time

# 기본 설정
plt.style.use('dark_background')  # 멋진 다크 테마
plt.rcParams['figure.facecolor'] = 'black'

class LaneKeepingSimulator:
    def __init__(self):
        # 도로 설정
        self.road_width = 100
        self.road_length = 200
        self.lane_width = 15

        # 차량 초기 위치
        self.car_x = 50  # 도로 중앙
        self.car_y = 20  # 시작 지점
        self.car_speed = 2

        # 차선 위치 (좌측, 중앙, 우측)
        self.left_lane = 35
        self.center_lane = 50
        self.right_lane = 65

        # 제어 변수
        self.steering_angle = 0
        self.target_lane = self.center_lane

        # 기록용
        self.positions = []
        self.steering_history = []

    def create_road(self):
        """도로와 차선 생성"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

        # 메인 시뮬레이션 화면
        ax1.set_xlim(0, self.road_width)
        ax1.set_ylim(0, 150)
        ax1.set_facecolor('black')
        ax1.set_title('🚗 Lane Keeping Simulation',
                     fontsize=16, color='white', fontweight='bold')
        ax1.set_xlabel('Road Width (m)', color='white')
        ax1.set_ylabel('Distance (m)', color='white')

        # 도로 그리기 (회색 아스팔트)
        road = plt.Rectangle((20, 0), 60, 150,
                           facecolor='#2C2C2C', edgecolor='white', linewidth=2)
        ax1.add_patch(road)

        # 차선 그리기 (하얀 점선)
        lane_y = np.arange(0, 150, 5)
        ax1.plot([self.left_lane]*len(lane_y), lane_y,
                'w--', linewidth=2, alpha=0.8, label='Left Lane')
        ax1.plot([self.center_lane]*len(lane_y), lane_y,
                'w--', linewidth=2, alpha=0.8, label='Center Lane')
        ax1.plot([self.right_lane]*len(lane_y), lane_y,
                'w--', linewidth=2, alpha=0.8, label='Right Lane')

        # 차량 (빨간 점으로 시작)
        self.car_dot, = ax1.plot(self.car_x, self.car_y, 'ro',
                                markersize=15, label='Vehicle')

        # 차량 경로 (초록 선)
        self.path_line, = ax1.plot([], [], 'g-', linewidth=3, alpha=0.7,
                                  label='Vehicle Path')

        # 범례
        ax1.legend(loc='upper right')
        ax1.grid(True, alpha=0.3)

        # 제어 정보 화면
        ax2.set_facecolor('black')
        ax2.set_title('📊 Control Information',
                     fontsize=16, color='white', fontweight='bold')
        ax2.set_xlim(0, 100)
        ax2.set_ylim(0, 100)

        # 실시간 정보 텍스트
        self.info_text = ax2.text(10, 80, '', fontsize=12, color='cyan',
                                 family='monospace')

        # 조향각 그래프
        self.steering_line, = ax2.plot([], [], 'y-', linewidth=2)
        ax2.set_xlabel('Time Steps', color='white')
        ax2.set_ylabel('Steering Angle', color='white')

        return fig, ax1, ax2

    def simple_controller(self):
        """간단한 PID 제어기"""
        # 목표 차선과의 거리 오차
        error = self.target_lane - self.car_x

        # 비례 제어 (P control)
        kp = 0.1  # 비례 상수
        self.steering_angle = kp * error

        # 조향각 제한 (-5도 ~ +5도)
        self.steering_angle = np.clip(self.steering_angle, -5, 5)

        return self.steering_angle

    def update_vehicle(self):
        """차량 위치 업데이트"""
        # 조향각에 따른 횡방향 이동
        lateral_movement = self.steering_angle * 0.3

        # 차량 위치 업데이트
        self.car_x += lateral_movement
        self.car_y += self.car_speed

        # 도로 경계 체크
        self.car_x = np.clip(self.car_x, 22, 78)

        # 경로 기록
        self.positions.append([self.car_x, self.car_y])
        self.steering_history.append(self.steering_angle)

    def animate(self, frame):
        """애니메이션 업데이트 함수"""
        # 제어기 실행
        steering = self.simple_controller()

        # 차량 업데이트
        self.update_vehicle()

        # 차량 위치 업데이트
        self.car_dot.set_data([self.car_x], [self.car_y])

        # 경로 표시
        if len(self.positions) > 1:
            path_x = [pos[0] for pos in self.positions[-20:]]  # 최근 20개점만
            path_y = [pos[1] for pos in self.positions[-20:]]
            self.path_line.set_data(path_x, path_y)

        # 제어 정보 업데이트
        info = f"""🚗 Vehicle Status:

Position X: {self.car_x:.1f} m
Position Y: {self.car_y:.1f} m
Speed: {self.car_speed:.1f} m/s
Steering: {self.steering_angle:.2f}°

Target Lane: {self.target_lane:.1f} m
Error: {self.target_lane - self.car_x:.2f} m

Frame: {frame}
        """
        self.info_text.set_text(info)

        # 조향각 히스토리 그래프
        if len(self.steering_history) > 1:
            self.steering_line.set_data(range(len(self.steering_history)),
                                      self.steering_history)

        # 화면 스크롤 (차량이 위로 올라가면)
        if self.car_y > 100:
            self.car_y = 20
            self.positions = []  # 경로 리셋

        return self.car_dot, self.path_line, self.info_text, self.steering_line

# 🎮 시뮬레이션 실행
print("🚗 Lane Keeping Simulation Starting...")
print("차량이 중앙 차선을 유지하며 주행합니다!")

# 시뮬레이터 생성
simulator = LaneKeepingSimulator()
fig, ax1, ax2 = simulator.create_road()

# 애니메이션 생성 (30프레임, 무한 반복)
anim = animation.FuncAnimation(
    fig,
    simulator.animate,
    interval=100,  # 100ms마다 업데이트
    frames=200,    # 200프레임
    blit=False,
    repeat=True
)

# Colab에서 애니메이션 표시
plt.tight_layout()
plt.show()

# 애니메이션을 HTML로 변환 (Colab에서 재생 가능)
print("\n🎬 애니메이션 생성 중... (잠시만 기다려주세요)")
html_anim = HTML(anim.to_jshtml())

print("✅ 시뮬레이션 완료!")
print("📊 차량이 PID 제어로 차선을 유지하며 주행했습니다!")

# 결과 통계
print(f"\n📈 시뮬레이션 결과:")
print(f"최대 조향각: {max(simulator.steering_history):.2f}°")
print(f"평균 조향각: {np.mean(simulator.steering_history):.2f}°")
print(f"주행 거리: {len(simulator.positions) * simulator.car_speed:.1f}m")

# HTML 애니메이션 반환
html_anim

영상저장 코드 추가

In [ ]:
# 🎬 완전한 차선 유지 시뮬레이션 + MP4 저장
# Google Colab에서 한 번에 실행하는 전체 코드

# ================================
# 1단계: 라이브러리 설치 및 설정
# ================================

print("📦 라이브러리 설치 및 설정 중...")

# 필요한 라이브러리 설치
import subprocess
import sys

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    except:
        pass

# 필수 패키지 설치
install_package("matplotlib")
install_package("numpy")
install_package("IPython")

# FFmpeg 설치 (애니메이션 저장용)
try:
    subprocess.run(["apt-get", "update"], check=True, capture_output=True)
    subprocess.run(["apt-get", "install", "-y", "ffmpeg"], check=True, capture_output=True)
    print("✅ FFmpeg 설치 완료!")
except:
    print("⚠️ FFmpeg 설치 건너뜀 (이미 설치되었거나 권한 없음)")

# 라이브러리 import
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display
import time
import warnings
warnings.filterwarnings('ignore')

# Colab 환경 설정
plt.style.use('dark_background')
plt.rcParams['figure.facecolor'] = 'black'
plt.rcParams['animation.html'] = 'jshtml'

print("✅ 모든 라이브러리 준비 완료!")

# ================================
# 2단계: 차선 유지 시뮬레이터 클래스
# ================================

class LaneKeepingSimulator:
    def __init__(self):
        """시뮬레이터 초기화"""
        # 도로 설정
        self.road_width = 100
        self.road_length = 200
        self.lane_width = 15

        # 차량 초기 위치
        self.car_x = 50  # 도로 중앙
        self.car_y = 20  # 시작 지점
        self.car_speed = 1.5

        # 차선 위치 (좌측, 중앙, 우측)
        self.left_lane = 35
        self.center_lane = 50
        self.right_lane = 65

        # 제어 변수
        self.steering_angle = 0
        self.target_lane = self.center_lane

        # 기록용
        self.positions = []
        self.steering_history = []
        self.time_steps = []

        print("🚗 시뮬레이터 초기화 완료!")

    def create_road(self):
        """도로와 차선 생성"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
        fig.patch.set_facecolor('black')

        # === 메인 시뮬레이션 화면 (왼쪽) ===
        ax1.set_xlim(0, self.road_width)
        ax1.set_ylim(0, 120)
        ax1.set_facecolor('black')
        ax1.set_title('🚗 Lane Keeping Simulation',
                     fontsize=18, color='white', fontweight='bold')
        ax1.set_xlabel('Road Width (m)', color='white', fontsize=12)
        ax1.set_ylabel('Distance (m)', color='white', fontsize=12)

        # 도로 그리기 (어두운 아스팔트)
        road = plt.Rectangle((20, 0), 60, 120,
                           facecolor='#1a1a1a', edgecolor='white', linewidth=3)
        ax1.add_patch(road)

        # 도로 경계선 (노란색)
        ax1.plot([20, 20], [0, 120], 'yellow', linewidth=4, alpha=0.8)
        ax1.plot([80, 80], [0, 120], 'yellow', linewidth=4, alpha=0.8)

        # 차선 그리기 (하얀 점선)
        lane_y = np.arange(0, 120, 8)
        for y in lane_y:
            # 왼쪽 차선
            ax1.plot([self.left_lane, self.left_lane], [y, y+4],
                    'white', linewidth=3, alpha=0.9)
            # 중앙 차선
            ax1.plot([self.center_lane, self.center_lane], [y, y+4],
                    'white', linewidth=3, alpha=0.9)
            # 오른쪽 차선
            ax1.plot([self.right_lane, self.right_lane], [y, y+4],
                    'white', linewidth=3, alpha=0.9)

        # 차량 (빨간 원으로 표시)
        self.car_dot, = ax1.plot(self.car_x, self.car_y, 'ro',
                                markersize=20, markeredgecolor='yellow',
                                markeredgewidth=2, label='Vehicle')

        # 차량 경로 (초록 선)
        self.path_line, = ax1.plot([], [], 'lime', linewidth=4, alpha=0.8,
                                  label='Vehicle Path')

        # 목표 차선 표시
        self.target_line, = ax1.plot([self.target_lane, self.target_lane], [0, 120],
                                    'cyan', linewidth=2, alpha=0.6, linestyle=':',
                                    label='Target Lane')

        # 범례
        ax1.legend(loc='upper right', fontsize=10)
        ax1.grid(True, alpha=0.2, color='gray')

        # === 제어 정보 화면 (오른쪽) ===
        ax2.set_facecolor('black')
        ax2.set_title('📊 Vehicle Control Dashboard',
                     fontsize=18, color='white', fontweight='bold')
        ax2.set_xlim(0, 100)
        ax2.set_ylim(0, 100)

        # 실시간 정보 텍스트
        self.info_text = ax2.text(5, 70, '', fontsize=13, color='cyan',
                                 family='monospace', verticalalignment='top')

        # 조향각 그래프 영역
        ax2.text(5, 40, 'Steering Angle History:', fontsize=12, color='white', fontweight='bold')
        self.steering_line, = ax2.plot([], [], 'yellow', linewidth=3, alpha=0.8)
        ax2.set_xlim(0, 100)
        ax2.set_ylim(30, 40)

        ax2.axis('off')  # 축 숨기기

        return fig, ax1, ax2

    def simple_controller(self):
        """PID 제어기 - 차선 유지"""
        # 목표 차선과의 거리 오차
        error = self.target_lane - self.car_x

        # P 제어 (비례 제어)
        kp = 0.15  # 비례 상수
        self.steering_angle = kp * error

        # 조향각 제한 (-3도 ~ +3도)
        max_steering = 3.0
        self.steering_angle = np.clip(self.steering_angle, -max_steering, max_steering)

        return self.steering_angle

    def update_vehicle(self, frame):
        """차량 위치 업데이트"""
        # 조향각에 따른 횡방향 이동
        lateral_movement = self.steering_angle * 0.2

        # 차량 위치 업데이트
        self.car_x += lateral_movement
        self.car_y += self.car_speed

        # 도로 경계 체크 (차량이 도로 밖으로 나가지 않도록)
        self.car_x = np.clip(self.car_x, 22, 78)

        # 화면 스크롤 (차량이 위로 올라가면 다시 아래로)
        if self.car_y > 100:
            self.car_y = 10
            # 경로는 유지 (연속적인 주행 효과)

        # 기록 저장
        self.positions.append([self.car_x, self.car_y])
        self.steering_history.append(self.steering_angle)
        self.time_steps.append(frame)

        # 최대 기록 수 제한 (메모리 절약)
        if len(self.positions) > 100:
            self.positions.pop(0)
            self.steering_history.pop(0)
            self.time_steps.pop(0)

    def animate(self, frame):
        """애니메이션 업데이트 함수"""
        # 제어기 실행
        steering = self.simple_controller()

        # 차량 업데이트
        self.update_vehicle(frame)

        # === 시각 요소 업데이트 ===

        # 1. 차량 위치 업데이트
        self.car_dot.set_data([self.car_x], [self.car_y])

        # 2. 경로 표시 (최근 30개 점)
        if len(self.positions) > 1:
            recent_positions = self.positions[-30:]
            path_x = [pos[0] for pos in recent_positions]
            path_y = [pos[1] for pos in recent_positions]
            self.path_line.set_data(path_x, path_y)

        # 3. 제어 정보 업데이트
        error = self.target_lane - self.car_x
        status_color = '🟢' if abs(error) < 1.0 else '🟡' if abs(error) < 2.0 else '🔴'

        info_text = f"""🚗 VEHICLE STATUS {status_color}

📍 Position X: {self.car_x:.1f} m
📍 Position Y: {self.car_y:.1f} m
🏃 Speed: {self.car_speed:.1f} m/s
🎯 Steering: {self.steering_angle:.2f}°

🛣️  Target Lane: {self.target_lane:.1f} m
📏 Lane Error: {error:.2f} m
⏱️  Frame: {frame}

{'🎯 PERFECT!' if abs(error) < 0.5 else '✅ Good' if abs(error) < 1.5 else '⚠️ Adjusting'}
        """
        self.info_text.set_text(info_text)

        # 4. 조향각 히스토리 (간단한 선 그래프)
        if len(self.steering_history) > 5:
            recent_steering = self.steering_history[-20:]
            x_data = list(range(len(recent_steering)))
            y_data = [35 + angle for angle in recent_steering]  # 35를 중심으로 표시
            self.steering_line.set_data(x_data, y_data)

        return self.car_dot, self.path_line, self.info_text, self.steering_line

# ================================
# 3단계: 시뮬레이션 실행
# ================================

print("\n🚗 Lane Keeping Simulation 시작!")
print("차량이 중앙 차선을 유지하며 주행합니다...")

# 시뮬레이터 생성
simulator = LaneKeepingSimulator()
fig, ax1, ax2 = simulator.create_road()

# 애니메이션 생성
print("🎬 애니메이션 생성 중...")
anim = animation.FuncAnimation(
    fig,
    simulator.animate,
    interval=150,    # 150ms마다 업데이트 (더 부드럽게)
    frames=300,      # 300프레임 (약 45초)
    blit=False,
    repeat=True
)

# 화면에 표시
plt.tight_layout()
plt.show()

print("✅ 시뮬레이션 실행 중!")

# ================================
# 4단계: MP4로 저장 및 다운로드
# ================================

def save_to_mp4(animation_obj, filename="lane_keeping_simulation"):
    """애니메이션을 MP4로 저장하고 다운로드"""

    print(f"\n💾 '{filename}.mp4' 저장 시작...")

    try:
        # MP4 writer 설정
        Writer = animation.writers['ffmpeg']
        writer = Writer(
            fps=8,                    # 초당 8프레임
            metadata=dict(
                artist='Lane Keeping AI',
                title='Autonomous Vehicle Lane Keeping Simulation',
                comment='Created with Python & Matplotlib'
            ),
            bitrate=1800             # 비트레이트 (화질)
        )

        # MP4 파일로 저장
        print("🎬 비디오 인코딩 중... (약 30초 소요)")
        animation_obj.save(f'{filename}.mp4', writer=writer, progress_callback=None)

        print("✅ MP4 저장 완료!")

        # 파일 다운로드
        print("📥 파일 다운로드 준비 중...")
        from google.colab import files
        files.download(f'{filename}.mp4')

        print("🎉 다운로드 완료! 파일을 확인해보세요!")

        # 파일 정보 표시
        import os
        file_size = os.path.getsize(f'{filename}.mp4') / (1024*1024)  # MB 단위
        print(f"📊 파일 크기: {file_size:.1f} MB")

    except Exception as e:
        print(f"❌ 저장 실패: {str(e)}")
        print("\n💡 대안 방법들:")
        print("1. 런타임을 재시작하고 다시 시도")
        print("2. 프레임 수를 줄여서 시도 (frames=100)")
        print("3. HTML 형태로 저장 시도")

        # HTML 대안 저장
        try:
            print("\n🌐 HTML 형태로 저장 시도 중...")
            with open(f'{filename}.html', 'w', encoding='utf-8') as f:
                f.write(anim.to_jshtml())
            files.download(f'{filename}.html')
            print("✅ HTML 파일로 저장 완료!")
        except:
            print("❌ HTML 저장도 실패")

# MP4 저장 실행
save_to_mp4(anim, "my_lane_keeping_simulation")

# ================================
# 5단계: 결과 리포트
# ================================

print("\n" + "="*50)
print("📊 SIMULATION REPORT")
print("="*50)

if len(simulator.steering_history) > 0:
    avg_steering = np.mean(np.abs(simulator.steering_history))
    max_steering = np.max(np.abs(simulator.steering_history))
    total_distance = len(simulator.positions) * simulator.car_speed

    print(f"🎯 총 주행 거리: {total_distance:.1f} m")
    print(f"📐 평균 조향각: {avg_steering:.2f}°")
    print(f"📐 최대 조향각: {max_steering:.2f}°")
    print(f"⏱️  총 시뮬레이션 프레임: {len(simulator.positions)}")
    print(f"🎬 예상 비디오 길이: {len(simulator.positions)/8:.1f}초")

    # 성능 평가
    if avg_steering < 1.0:
        print("🏆 성능 평가: 우수! (안정적인 차선 유지)")
    elif avg_steering < 2.0:
        print("✅ 성능 평가: 양호 (적절한 차선 유지)")
    else:
        print("⚠️ 성능 평가: 개선 필요 (조향이 불안정)")

print("\n🎉 모든 작업 완료!")
print("💡 비디오 파일을 다운로드 받아서 친구들에게 자랑해보세요!")
print("🚀 다음에는 곡선 도로나 장애물 회피 기능을 추가해볼까요?")

새로운 곡선 도로 기능들
1. S자 커브 도로

🌊 복잡한 곡선: 큰 커브 + 작은 커브 + 세밀한 커브
📏 실시간 곡률 계산: 도로 상황에 따른 동적 대응

2. 고급 PID 제어

🔮 미래 예측: 15m 앞 도로 상황 미리 파악
⚖️ PID 3요소: 비례(P) + 적분(I) + 미분(D) 제어
🎯 적응적 튜닝: 도로 상황에 따른 파라미터 조정

3. 물리 시뮬레이션

🚗 Bicycle Model: 실제 차량 동역학 적용
🧭 방향각 추적: 차량 헤딩 고려
⚡ 실시간 물리: 조향각 → 경로 변화

4. 4분할 대시보드

🛣️ 곡선 도로 시각화
📊 실시간 제어 정보
📈 조향각 히스토리
🏃 속도 & 곡률 그래프

In [ ]:
# 첫 번째 셀 - 라이브러리 설치
!pip install matplotlib
!pip install numpy
!pip install IPython

# 애니메이션이 안 되면 추가 설치
!apt-get update
!apt-get install -y ffmpeg

1. 코드 실행 ▶️
2. 라이브러리 설치 📦
3. 시뮬레이션 화면 표시 🎮
4. 실시간 애니메이션 재생 🎬
5. 백그라운드에서 MP4 인코딩 🔄
6. 다운로드 팝업 자동 등장 📥
7. 완료 메시지 출력 ✅

In [ ]:
# 🌊 곡선 도로 차선 유지 시뮬레이션
# 실제 하드웨어 적용 가능한 제어 로직 포함

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import math

# 설정
plt.style.use('dark_background')
plt.rcParams['figure.facecolor'] = 'black'

class CurvedRoadSimulator:
    def __init__(self):
        """곡선 도로 시뮬레이터 초기화"""
        # 기본 설정
        self.road_width = 100
        self.car_x = 50
        self.car_y = 10
        self.car_speed = 1.2
        self.car_heading = 0  # 차량 방향각 추가

        # 제어 변수 (실제 차량에서도 동일하게 사용 가능)
        self.steering_angle = 0
        self.max_steering = 25  # 최대 조향각 (도)

        # PID 제어 파라미터 (실제 차량 튜닝 가능)
        self.kp = 0.8   # 비례 게인
        self.ki = 0.01  # 적분 게인
        self.kd = 0.3   # 미분 게인

        # PID 제어용 변수들
        self.previous_error = 0
        self.integral_error = 0

        # 기록용
        self.positions = []
        self.steering_history = []
        self.speed_history = []
        self.road_curvature_history = []

        print("🌊 곡선 도로 시뮬레이터 준비 완료!")

    def get_road_curvature(self, y_position):
        """도로의 곡률 계산 - S자 커브"""
        # S자 곡선 생성 (사인파 조합)
        curve1 = 15 * np.sin(y_position * 0.08)      # 큰 커브
        curve2 = 8 * np.sin(y_position * 0.15 + 1)   # 작은 커브
        curve3 = 5 * np.sin(y_position * 0.25 + 2)   # 세밀한 커브

        total_curvature = curve1 + curve2 + curve3
        return total_curvature

    def get_lane_positions(self, y_position):
        """현재 Y 위치에서의 차선 위치들"""
        center_offset = self.get_road_curvature(y_position)
        base_center = 50

        # 3개 차선 (좌, 중, 우)
        left_lane = base_center + center_offset - 15
        center_lane = base_center + center_offset
        right_lane = base_center + center_offset + 15

        return left_lane, center_lane, right_lane

    def get_target_lane(self, y_position):
        """목표 차선 (현재는 중앙차선, 실제로는 경로 계획에서 결정)"""
        _, center_lane, _ = self.get_lane_positions(y_position)
        return center_lane

    def advanced_pid_controller(self):
        """고급 PID 제어 (실제 차량에서 사용 가능한 로직)"""
        # 현재 목표 차선
        target_lane = self.get_target_lane(self.car_y)

        # 미래 예측 (lookahead) - 실제 차량에서 중요!
        lookahead_distance = 15
        future_target = self.get_target_lane(self.car_y + lookahead_distance)

        # 현재 오차와 미래 오차 고려
        current_error = target_lane - self.car_x
        future_error = future_target - self.car_x

        # PID 계산
        # P: 비례 제어 (현재 오차)
        proportional = self.kp * current_error

        # I: 적분 제어 (누적 오차) - 정상상태 오차 제거
        self.integral_error += current_error * 0.1  # dt = 0.1
        integral = self.ki * self.integral_error

        # D: 미분 제어 (오차 변화율) - 오버슈트 방지
        error_derivative = (current_error - self.previous_error) / 0.1
        derivative = self.kd * error_derivative

        # PID 출력
        pid_output = proportional + integral + derivative

        # 미래 예측을 반영한 추가 조정
        curvature_compensation = (future_error - current_error) * 0.3

        # 최종 조향각 계산
        self.steering_angle = pid_output + curvature_compensation

        # 조향각 제한 (실제 차량의 물리적 한계)
        self.steering_angle = np.clip(self.steering_angle, -self.max_steering, self.max_steering)

        # 다음 계산을 위해 저장
        self.previous_error = current_error

        return self.steering_angle, current_error, target_lane

    def adaptive_speed_control(self):
        """적응적 속도 제어 - 커브에서 감속"""
        # 도로 곡률에 따른 속도 조정
        curvature = abs(self.get_road_curvature(self.car_y + 10))  # 앞쪽 곡률 예측

        # 곡률이 클수록 속도 감소
        if curvature > 20:
            target_speed = 0.8    # 급커브: 감속
        elif curvature > 10:
            target_speed = 1.0    # 보통커브: 약간 감속
        else:
            target_speed = 1.2    # 직선: 정상 속도

        # 부드러운 속도 변화
        speed_diff = target_speed - self.car_speed
        self.car_speed += speed_diff * 0.1  # 점진적 변화

        return self.car_speed

    def update_vehicle_physics(self):
        """차량 물리 시뮬레이션 (실제 차량 동역학 근사)"""
        # 조향각을 라디안으로 변환
        steering_rad = np.radians(self.steering_angle)

        # 차량 방향각 업데이트 (bicycle model 근사)
        wheelbase = 2.5  # 차축거리 (m)
        self.car_heading += (self.car_speed * np.tan(steering_rad) / wheelbase) * 0.1

        # 위치 업데이트
        self.car_x += self.car_speed * np.sin(self.car_heading) * 0.1
        self.car_y += self.car_speed * np.cos(self.car_heading) * 0.1

        # 화면 경계 처리
        if self.car_y > 120:
            self.car_y = 10
            self.integral_error = 0  # 적분 오차 리셋

        # 도로 이탈 방지 (안전 기능)
        left_boundary = 15
        right_boundary = 85
        if self.car_x < left_boundary:
            self.car_x = left_boundary
            self.car_heading = 0  # 방향 보정
        elif self.car_x > right_boundary:
            self.car_x = right_boundary
            self.car_heading = 0

    def create_curved_road_visualization(self):
        """곡선 도로 시각화"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 12))
        fig.patch.set_facecolor('black')

        # === 메인 시뮬레이션 (좌상) ===
        ax1.set_xlim(0, 100)
        ax1.set_ylim(0, 120)
        ax1.set_facecolor('black')
        ax1.set_title('🌊 Curved Road Lane Keeping', fontsize=16, color='white', fontweight='bold')

        # 곡선 도로 그리기
        y_road = np.linspace(0, 120, 200)
        road_curves = [self.get_road_curvature(y) for y in y_road]

        # 도로 중앙선과 차선들
        center_line = [50 + curve for curve in road_curves]
        left_line = [center - 15 for center in center_line]
        right_line = [center + 15 for center in center_line]

        # 도로 경계 (진한 회색)
        ax1.fill_betweenx(y_road, [c-25 for c in center_line], [c+25 for c in center_line],
                         color='#2a2a2a', alpha=0.8, label='Road')

        # 차선 (점선)
        ax1.plot(left_line, y_road, 'w--', linewidth=2, alpha=0.7, label='Left Lane')
        ax1.plot(center_line, y_road, 'w--', linewidth=2, alpha=0.7, label='Center Lane')
        ax1.plot(right_line, y_road, 'w--', linewidth=2, alpha=0.7, label='Right Lane')

        # 차량과 경로
        self.car_dot, = ax1.plot(self.car_x, self.car_y, 'ro', markersize=15,
                                markeredgecolor='yellow', markeredgewidth=2)
        self.path_line, = ax1.plot([], [], 'lime', linewidth=3, alpha=0.8)
        self.target_line, = ax1.plot([], [], 'cyan', linewidth=2, alpha=0.6, linestyle=':')

        ax1.legend(loc='upper right')
        ax1.grid(True, alpha=0.2)

        # === 제어 대시보드 (우상) ===
        ax2.set_facecolor('black')
        ax2.set_title('🎮 Control Dashboard', fontsize=16, color='white', fontweight='bold')
        ax2.set_xlim(0, 10)
        ax2.set_ylim(0, 10)

        self.dashboard_text = ax2.text(0.5, 5, '', fontsize=11, color='cyan',
                                      family='monospace', ha='left', va='center')
        ax2.axis('off')

        # === 조향각 히스토리 (좌하) ===
        ax3.set_facecolor('black')
        ax3.set_title('📈 Steering Angle History', fontsize=14, color='white', fontweight='bold')
        ax3.set_xlabel('Time Steps', color='white')
        ax3.set_ylabel('Steering Angle (°)', color='white')
        ax3.grid(True, alpha=0.3)

        self.steering_plot, = ax3.plot([], [], 'yellow', linewidth=2)
        ax3.set_xlim(0, 100)
        ax3.set_ylim(-30, 30)

        # === 속도 및 곡률 (우하) ===
        ax4.set_facecolor('black')
        ax4.set_title('🏃 Speed & Road Curvature', fontsize=14, color='white', fontweight='bold')
        ax4.set_xlabel('Time Steps', color='white')
        ax4.set_ylabel('Values', color='white')
        ax4.grid(True, alpha=0.3)

        self.speed_plot, = ax4.plot([], [], 'green', linewidth=2, label='Speed (m/s)')
        self.curvature_plot, = ax4.plot([], [], 'orange', linewidth=2, label='Curvature/10')
        ax4.legend()
        ax4.set_xlim(0, 100)
        ax4.set_ylim(0, 3)

        return fig, ax1, ax2, ax3, ax4

    def animate(self, frame):
        """애니메이션 업데이트"""
        # 1. 제어 시스템 실행
        steering, error, target = self.advanced_pid_controller()
        speed = self.adaptive_speed_control()

        # 2. 차량 물리 업데이트
        self.update_vehicle_physics()

        # 3. 데이터 기록
        self.positions.append([self.car_x, self.car_y])
        self.steering_history.append(steering)
        self.speed_history.append(speed)
        self.road_curvature_history.append(abs(self.get_road_curvature(self.car_y))/10)

        # 기록 길이 제한
        max_history = 100
        if len(self.positions) > max_history:
            self.positions.pop(0)
            self.steering_history.pop(0)
            self.speed_history.pop(0)
            self.road_curvature_history.pop(0)

        # 4. 시각화 업데이트
        # 차량 위치
        self.car_dot.set_data([self.car_x], [self.car_y])

        # 차량 경로
        if len(self.positions) > 5:
            path_x = [pos[0] for pos in self.positions[-30:]]
            path_y = [pos[1] for pos in self.positions[-30:]]
            self.path_line.set_data(path_x, path_y)

        # 목표 차선 표시
        target_y = np.linspace(max(0, self.car_y-10), min(120, self.car_y+30), 50)
        target_x = [self.get_target_lane(y) for y in target_y]
        self.target_line.set_data(target_x, target_y)

        # 대시보드 업데이트
        curvature = abs(self.get_road_curvature(self.car_y))
        road_type = "🏁 Straight" if curvature < 5 else "🌊 Curve" if curvature < 15 else "⚡ Sharp Curve"

        dashboard_info = f"""🚗 VEHICLE STATUS

📍 Position: ({self.car_x:.1f}, {self.car_y:.1f})
🧭 Heading: {np.degrees(self.car_heading):.1f}°
🏃 Speed: {speed:.2f} m/s
🎯 Steering: {steering:.1f}°

🛣️ ROAD CONDITIONS
{road_type}
📏 Lane Error: {error:.2f} m
📐 Curvature: {curvature:.1f}

⚙️ CONTROL STATUS
P: {self.kp * error:.2f}
I: {self.ki * self.integral_error:.2f}
D: {self.kd * (error - self.previous_error)/0.1:.2f}

⏱️ Frame: {frame}
        """
        self.dashboard_text.set_text(dashboard_info)

        # 그래프 업데이트
        if len(self.steering_history) > 1:
            time_steps = list(range(len(self.steering_history)))
            self.steering_plot.set_data(time_steps, self.steering_history)
            self.speed_plot.set_data(time_steps, self.speed_history)
            self.curvature_plot.set_data(time_steps, self.road_curvature_history)

        return (self.car_dot, self.path_line, self.target_line, self.dashboard_text,
                self.steering_plot, self.speed_plot, self.curvature_plot)

# === 실행 ===
print("🌊 Curved Road Simulation 시작!")
print("S자 커브에서 고급 PID 제어로 차선 유지!")

simulator = CurvedRoadSimulator()
fig, ax1, ax2, ax3, ax4 = simulator.create_curved_road_visualization()

# 애니메이션 생성
anim = animation.FuncAnimation(
    fig, simulator.animate,
    interval=100, frames=400, blit=False, repeat=True
)

plt.tight_layout()
plt.show()

# MP4 저장 기능
def save_curved_road_mp4():
    print("\n💾 곡선 도로 시뮬레이션 MP4 저장 중...")
    try:
        Writer = animation.writers['ffmpeg']
        writer = Writer(fps=10, bitrate=2000)
        anim.save('curved_road_simulation.mp4', writer=writer)

        from google.colab import files
        files.download('curved_road_simulation.mp4')
        print("✅ 곡선 도로 비디오 저장 완료!")
    except Exception as e:
        print(f"❌ 저장 실패: {e}")

# 저장 실행
save_curved_road_mp4()

print("\n🎯 실제 차량 적용 가능한 핵심 코드:")
print("- PID 제어 로직: simulator.advanced_pid_controller()")
print("- 속도 제어: simulator.adaptive_speed_control()")
print("- 차량 동역학: simulator.update_vehicle_physics()")
print("🚀 이 알고리즘들은 실제 자율주행차에서 그대로 사용 가능!")